In [97]:
import pandas as pd 
import matplotlib.pyplot as plt
from scipy.stats import entropy
import numpy as np
import altair as alt 
import plotly.graph_objects as go
import seaborn as sns
import networkx as nx

In [98]:
# load in data from our 3 California national Parks 
df_redwood = pd.read_excel("redwood.xlsx")
df_joshua = pd.read_excel("joshua.xlsx")
df_sequoia = pd.read_excel("sequoia.xlsx")

# combine them into one dataframe
df_combined = pd.concat([df_redwood, df_joshua, df_sequoia], ignore_index=True)
df_combined.shape

(10661, 33)

In [99]:
# look at missing values by column
df_combined.isnull().sum()

Region                 10661
Park Code                  0
Park Name                  0
Category                   0
Category Sort              0
Order                    500
Family                   504
Taxon Code                 0
TSN                       47
Taxon Record Status        0
Scientific Name            0
Common Names            4166
Synonyms                9954
Park Accepted              0
Sens?                      0
Record Status              0
Occurrence              5315
Occurrence Tags        10563
Nativeness              5324
Nativeness Tags        10651
Abundance               6376
NPS Tags                9255
Park Tags              10636
Internal Park Tags     10661
References               678
Observations            9455
Vouchers                7034
ExternalLinks          10661
T&E                     9814
State Status           10463
Ozone                  10614
GRank                   4809
SRank                   6005
dtype: int64

In [100]:
# drop columns with large amount of missing values
df_combined = df_combined.drop(['Region', 'Internal Park Tags', 'ExternalLinks', 'Ozone', 'Park Tags', 'Nativeness Tags', 'Occurrence Tags'], axis=1)
df_combined.shape

(10661, 26)

In [101]:
# drop species not in park or unconfirmed
df_combined = df_combined[~df_combined['Occurrence'].isin(['Not In Park', 'Unconfirmed'])]
df_combined.shape

(10070, 26)

In [ ]:
# drop species with no family/order name
df_combined = df_combined[~df_combined['Order'].isnull()]
df_combined = df_combined[~df_combined['Family'].isnull()]
df_combined = df_combined[df_combined['Record Status'].isin(['Approved'])]

# Filter out rows with more than two words in the 'Scientific Name' column
df_combined = df_combined[df_combined['Scientific Name'].str.split().str.len() == 2]

# Split the 'Scientific Name' column into 'Genus' and 'Species'
df_combined[['Genus', 'Species']] = df_combined['Scientific Name'].str.split(' ', expand=True)

# get just non-native yet abundant species
non_native_common = df_combined[(df_combined['Nativeness'] == 'Non-native') & 
                                (df_combined['Abundance'].isin(['Abundant', 'Common']))]


In [169]:
# Step 1: Get the counts for order -> family
category_order_count = non_native_common.groupby(['Category', 'Order']).size().reset_index(name='count')

# Step 2: Get the counts for family -> genus
order_park_count = non_native_common.groupby(['Order', 'Park Name']).size().reset_index(name='count')

# Combine the order -> family and family -> genus into one list of source, target, and value for Sankey
# For order -> family
order_family_links = category_order_count[['Category', 'Order', 'count']].rename(columns={'Category': 'source', 'Order': 'target', 'count': 'value'})

# For family -> genus
family_genus_links = order_park_count[['Order', 'Park Name', 'count']].rename(columns={'Order': 'source', 'Park Name': 'target', 'count': 'value'})

# Concatenate the two link DataFrames
links = pd.concat([order_family_links, family_genus_links], ignore_index=True)

# Create a list of nodes
nodes = list(pd.concat([category_order_count['Category'], category_order_count['Order'], order_park_count['Park Name']]).unique())

# Map the nodes to indices
nodes_dict = {node: i for i, node in enumerate(nodes)}

# Add node indices to the links
links['source'] = links['source'].map(nodes_dict)
links['target'] = links['target'].map(nodes_dict)

# Create a color map for nodes
order_color = 'blue'
family_color = 'green'
genus_color = 'red'

# Assign colors to nodes
node_colors = []
for node in nodes:
    if node in category_order_count['Category'].values:
        node_colors.append(order_color)
    elif node in order_park_count['Order'].values:
        node_colors.append(family_color)
    else:
        node_colors.append(genus_color)

# Create the Sankey diagram
fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=nodes,
        color=node_colors 
    ),
    link=dict(
        source=links['source'],
        target=links['target'],
        value=links['value']
    )
))

fig.update_layout(title_text="Non-Native Abundant Species", font_size=14, height=800)
fig.show()
fig.write_html("sankey.html")


In [165]:
redwood = df_combined[(df_combined['Park Code'] == 'REDW')]

# Step 1: Get the counts for order -> family
order_family_count = redwood.groupby(['Order', 'Family']).size().reset_index(name='count')

# Step 2: Get the counts for family -> genus
family_genus_count = redwood.groupby(['Family', 'Genus']).size().reset_index(name='count')

# Combine the order -> family and family -> genus into one list of source, target, and value for Sankey
# For order -> family
order_family_links = order_family_count[['Order', 'Family', 'count']].rename(columns={'Order': 'source', 'Family': 'target', 'count': 'value'})

# For family -> genus
family_genus_links = family_genus_count[['Family', 'Genus', 'count']].rename(columns={'Family': 'source', 'Genus': 'target', 'count': 'value'})

# Concatenate the two link DataFrames
links = pd.concat([order_family_links, family_genus_links], ignore_index=True)

# Create a list of nodes
nodes = list(pd.concat([order_family_count['Order'], order_family_count['Family'], family_genus_count['Genus']]).unique())

# Map the nodes to indices
nodes_dict = {node: i for i, node in enumerate(nodes)}

# Add node indices to the links
links['source'] = links['source'].map(nodes_dict)
links['target'] = links['target'].map(nodes_dict)

# Create the Sankey diagram
fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=nodes
    ),
    link=dict(
        source=links['source'],
        target=links['target'],
        value=links['value']
    )
))

fig.update_layout(title_text="redwood", font_size=14, height=800)
fig.show()

In [166]:
seqouia = df_combined[(df_combined['Park Code'] == 'SEKI')]

# Step 1: Get the counts for order -> family
order_family_count = seqouia.groupby(['Order', 'Family']).size().reset_index(name='count')

# Step 2: Get the counts for family -> genus
family_genus_count = seqouia.groupby(['Family', 'Genus']).size().reset_index(name='count')

# Combine the order -> family and family -> genus into one list of source, target, and value for Sankey
# For order -> family
order_family_links = order_family_count[['Order', 'Family', 'count']].rename(columns={'Order': 'source', 'Family': 'target', 'count': 'value'})

# For family -> genus
family_genus_links = family_genus_count[['Family', 'Genus', 'count']].rename(columns={'Family': 'source', 'Genus': 'target', 'count': 'value'})

# Concatenate the two link DataFrames
links = pd.concat([order_family_links, family_genus_links], ignore_index=True)

# Create a list of nodes
nodes = list(pd.concat([order_family_count['Order'], order_family_count['Family'], family_genus_count['Genus']]).unique())

# Map the nodes to indices
nodes_dict = {node: i for i, node in enumerate(nodes)}

# Add node indices to the links
links['source'] = links['source'].map(nodes_dict)
links['target'] = links['target'].map(nodes_dict)

# Create the Sankey diagram
fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=nodes
    ),
    link=dict(
        source=links['source'],
        target=links['target'],
        value=links['value']
    )
))

fig.update_layout(title_text="sequoia", font_size=14, height=800)
fig.show()

In [167]:
joshua = df_combined[(df_combined['Park Code'] == 'JOTR')]

# Step 1: Get the counts for order -> family
order_family_count = joshua.groupby(['Order', 'Family']).size().reset_index(name='count')

# Step 2: Get the counts for family -> genus
family_genus_count = joshua.groupby(['Family', 'Genus']).size().reset_index(name='count')

# Combine the order -> family and family -> genus into one list of source, target, and value for Sankey
# For order -> family
order_family_links = order_family_count[['Order', 'Family', 'count']].rename(columns={'Order': 'source', 'Family': 'target', 'count': 'value'})

# For family -> genus
family_genus_links = family_genus_count[['Family', 'Genus', 'count']].rename(columns={'Family': 'source', 'Genus': 'target', 'count': 'value'})

# Concatenate the two link DataFrames
links = pd.concat([order_family_links, family_genus_links], ignore_index=True)

# Create a list of nodes
nodes = list(pd.concat([order_family_count['Order'], order_family_count['Family'], family_genus_count['Genus']]).unique())

# Map the nodes to indices
nodes_dict = {node: i for i, node in enumerate(nodes)}

# Add node indices to the links
links['source'] = links['source'].map(nodes_dict)
links['target'] = links['target'].map(nodes_dict)

# Create the Sankey diagram
fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=nodes
    ),
    link=dict(
        source=links['source'],
        target=links['target'],
        value=links['value']
    )
))

fig.update_layout(title_text="joshua", font_size=14, height=800)
fig.show()